In [1]:
import requests
import json
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [46]:
source = 'dailyheadlines'
month = '02'
days = [str(i) for i in list(range(18,29))]

dates = []
for day in days:
    dates.append("{0}-{1}".format(month, day))

In [56]:
for date in dates:
    # Load HTML
    filepath = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/{1}.html'.format(source,date)
    with open(filepath) as f:
        soup = BeautifulSoup(f, 'lxml')
    
    # Pull out article links
    links = []
    i=0

    for li in soup.findAll('li', attrs={'class':'infinite-post'}):
        for link in li.findAll('a'):
            links.append(link.get('href'))

    links = set(links)
    links = list(links)


    # Grab content from articles
    i=0
    for link in links:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')
        title = soup.title.string.replace(' – Daily Headlines', '')
        try:
            pubdate = soup.find('time', attrs={'class':'post-date updated'})["datetime"]
        except TypeError:
            pubdate=''

        if date in pubdate:
            date_longform = '{0}-2017'.format(date)
            contents = []
            for div in soup.find('div', attrs={"id":"content-main"}):
                contents.append(div)

            article = {}
            content = ''.join(str(p) for p in contents)
            article["title"] = title
            article["url"] = link
            article["date"] = date_longform
            article["source"] = source
            article["content"] = content

            outfile = '/mnt/c/Users/Brennan/Dropbox/MIDS/w210-capstone/data/notCredible/{0}/articles/{2}/article{1}_{2}.txt'.format(source,i,date)
            os.makedirs(os.path.dirname(outfile), exist_ok=True)
            i+=1

            with open(outfile, 'w') as f:
                json.dump(article, f)